In [1]:
import numpy as np
from scipy.spatial import distance_matrix

In [2]:
from data_cube import DataCube

In [3]:
dc = DataCube(subjects=["10", "20"],
              gestures="all",
              channels="all",
              data_grp="parsed")
dc.load_data()

In [6]:
dc.data_set["10"]["5_0_2"].shape

(1763, 10)

---

In [ ]:
dc = DataCube(subjects=["10", "20"],
              gestures="all",
              channels=["2", "4", "5", "6", "8"],
              data_grp="master")
dc.load_data()

In [ ]:
# Create SSMs

class SSM:
    
    def __init__(self, time_series, metric):
        """
        CURRENTLY SUPPORTS SCALARS
        time series - (t x 1+m) numpy array where t is time series and 1+m is time index + number of modalities
        metric - metric to be used in generating SSMS
        """
        self.tidx = time_series[:, 0] # time index
        self.mods = time_series[:, 1:] # modalities
        self.metric = metric
        self.n_obs = self.tidx.size # number of obs
        self.n_mods = self.mods.shape[1] # number of modalitites
        self.array_dict = np.zeros(shape=(self.n_mods, self.n_obs, self.n_obs),dtype=np.float64) # tensor
        
        
    def calc_SSM(self):
        """
        calculate SSM
        """
        for m in range(self.n_mods): # loop over modalities
            for i in range(self.n_obs): # loop over observations in m
                for j in range(self.n_obs):
                    if i < j: # fill lower triangle only
                        self.array[m, i, j] = metric(self.mods[i, m], self.mods[j, m])
            self.array[m, :, :] = self.array[m, :, :] + self.array[m, :, :].T
                    
        
        
        
    



In [ ]:
D = SSM(dc.data_set["10"]["1"][:-1], lambda i,j:abs(i-j))
D.calc_SSM()